In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder.appName("Pride and Prejudice Word Count").getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/10/16 19:03:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import pyspark.sql.functions as F
import os
import pandas as pd

In [5]:
data_path = "./to_count/*.txt"

In [6]:
result = (spark.read.text(data_path).
          select(F.lower(F.col("value")).alias("lower_word")).
          select(F.split(F.col("lower_word"), " ").alias("split_word")).
          select(F.explode(F.col("split_word")).alias("explode_word")).
          select(F.regexp_extract(F.col("explode_word"), "[a-z]*", 0).alias("extract_word")).
          where((F.col("extract_word") != "") & (F.length(F.col("extract_word")) > 3)).
          groupby(F.col("extract_word")).
          count()
)

result.coalesce(1).write.csv("./combined_words5.csv") 